In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from google.colab import files
from io import BytesIO
from PIL import Image
import numpy as np

# Define directory paths for training and validation data
train_dir = '/content/drive/MyDrive/DataSet/train'
valid_dir = '/content/drive/MyDrive/DataSet/valid'

# Define ImageDataGenerator for data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size and image size
batch_size = 32
image_size = (224, 224)

# Generate batches of augmented data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load MobileNetV2 pre-trained model without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)

# Define the number of classes
num_classes = len(train_generator.class_indices)

predictions = Dense(num_classes, activation='softmax')(x)

# Combine base model with custom head
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=10
)

# Save the model
model.save('/content/drive/MyDrive/model/flower_classification_model.h5')

# Load your trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/model/flower_classification_model.h5')

# Load class labels
class_labels = list(train_generator.class_indices.keys())

# Function to preprocess and predict on user-uploaded image
def predict_uploaded_image(model, class_labels, image_size=(224, 224)):
    # Upload image file
    uploaded = files.upload()
    for filename in uploaded.keys():
        # Load and resize the image while maintaining aspect ratio
        img = Image.open(BytesIO(uploaded[filename]))
        img.thumbnail(image_size)

        # Convert image to array and preprocess
        img_array = np.array(img)
        img_array = img_array / 255.0  # Rescale pixel values

        # Expand dimensions to create a batch
        img_array = np.expand_dims(img_array, axis=0)

        # Standardize the shape of the input batch
        img_array = tf.image.resize_with_pad(img_array, image_size[0], image_size[1])

        # Make prediction
        prediction = model.predict(img_array)
        confidence = np.max(prediction)
        predicted_class_idx = np.argmax(prediction)

        # Define a confidence threshold
        confidence_threshold = 0.6

        if confidence < confidence_threshold:
            print("The model is not confident in its prediction. This image may not be part of the trained categories.")
        else:
            predicted_class = class_labels[predicted_class_idx]
            print(f"Predicted class: {predicted_class} (index: {predicted_class_idx}) with confidence: {confidence:.2f}")

    # Print the shape of the output
    print("Output shape:", prediction.shape)

# Call function to predict on user-uploaded image with dynamic size handling
predict_uploaded_image(model, class_labels)



Found 82 images belonging to 5 classes.
Found 36 images belonging to 5 classes.


9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/10
2/2 [==============================] - 60s 31s/step - loss: 1.4354 - accuracy: 0.3906 - val_loss: 0.9335 - val_accuracy: 0.6250
Epoch 2/10
2/2 [==============================] - 22s 10s/step - loss: 0.5161 - accuracy: 0.8400 - val_loss: 0.4456 - val_accuracy: 0.8125
Epoch 3/10
2/2 [==============================] - 21s 15s/step - loss: 0.1692 - accuracy: 1.0000 - val_loss: 0.2281 - val_accuracy: 0.8438
Epoch 4/10
2/2 [==============================] - 21s 16s/step - loss: 0.2143 - accuracy: 0.9200 - val_loss: 0.3676 - val_accuracy: 0.8125
Epoch 5/10
2/2 [==============================] - 20s 10s/step - loss: 0.1104 - accuracy: 0.9200 - val_loss: 0.2309 - val_accuracy: 0.9062
Epoch 6/10
2/2 [==============================] - 25s 15s/step - loss: 0.0305 - accuracy: 1.0000 - val_loss: 0.2331 - val_accuracy: 0.9062
Epoch 7/10
2/2 [==============================] - 21s 10s/step - loss: 0.0986 - accuracy: 0.9600 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saving pexels-picjumbo-com-55570-196664.jpg to pexels-picjumbo-com-55570-196664.jpg
1/1 [==============================] - 1s 939ms/step
Predicted class: Rose (index: 2) with confidence: 0.98
Output shape: (1, 5)


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from google.colab import files
from io import BytesIO
from PIL import Image

# Define image size
image_size = (224, 224)

# Load your trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/model/flower_classification_model.h5')

# Function to preprocess and predict on user-uploaded image
def predict_uploaded_image(model):
    # Upload image file
    uploaded = files.upload()
    for filename in uploaded.keys():
        # Load and preprocess the image
        img = Image.open(BytesIO(uploaded[filename])).resize(image_size)
        img_array = img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Create batch axis
        img_array /= 255.  # Rescale pixel values

        # Make prediction
        prediction = model.predict(img_array)
        predicted_class = list(train_generator.class_indices.keys())[prediction.argmax()]

        print(f"Predicted class: {predicted_class}")

# Call function to predict on user-uploaded image
predict_uploaded_image(model)


Saving images.jfif to images.jfif
1/1 [==============================] - 1s 875ms/step
Predicted class: Rose


In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import os
os.getcwd()


# Load your trained model
model = load_model('/content/drive/MyDrive/model/flower_classification_model.h5')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to file
with open('/content/drive/MyDrive/model/flower_classification_model.tflite', 'wb') as f:
    f.write(tflite_model)
